In [ ]:
import os
workingDirInit = os.getcwd()
workingDir = workingDirInit
parentDir = ((os.path.dirname(os.path.dirname(os.path.abspath(''))))) # ../../../
print('working dir ' + workingDir)
print('path dir ' + parentDir)

In [ ]:
import sys
sys.path
sys.path.append(parentDir)
print(sys.path)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# %run -i "rcpsp_calendar_examples.py"
import logging
import os
import glob
import time
import nest_asyncio
from skdecide import rollout_episode

from Code.rcpsp_calendar import parse_file
from Code.rcpsp_calendar_utils import (plot_resource_individual_gantt,
                                       plot_ressource_view, plot_task_gantt,
                                       save_task_gantt_resolution)
from Code.rcpsp_datasets import get_complete_path

from discrete_optimization.generic_tools.ea.ga import (
    DeapCrossover,
    DeapMutation,
    Ga,
    ObjectiveHandling,
)


nest_asyncio.apply()


logging.basicConfig(level=logging.INFO)

fig1 =None
fig2 =None
fig3 =None

In [ ]:
# def do_singlemode_ga_calendar(file_name):
#     global fig1, fig2, fig3
file_name = "j301_1_calendar.sm"
problem = parse_file(get_complete_path(file_name))

mutation = DeapMutation.MUT_SHUFFLE_INDEXES
crossover = DeapCrossover.CX_UNIFORM_PARTIALY_MATCHED
ga_solver = Ga(
    problem,
    encoding="rcpsp_permutation",
    objective_handling=ObjectiveHandling.AGGREGATE,
    objectives=["makespan"],
    objective_weights=[-1],
    pop_size=50,
    max_evals=300,
    mut_rate=0.1,
    crossover_rate=0.9,
    crossover=crossover,
    mutation=mutation,
)
results_ga = ga_solver.solve()
solution = results_ga.get_best_solution_fit()
solution_gantt_df = save_task_gantt_resolution(problem, solution[0])
print(solution_gantt_df)


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook
fig1 = plot_task_gantt(problem, solution[0], title=file_name, fig=fig1)

In [ ]:
%matplotlib notebook
# Create the figure and axis
fig2 = plot_ressource_view(problem, solution[0],
                           title_figure=file_name,
                           fig=fig2)


In [ ]:
%matplotlib notebook
fig3 = plot_resource_individual_gantt(problem, solution[0],
                                      title_figure=file_name,
                                      fig=fig3)


In [ ]:
fig3.savefig("filepath.svg", format = 'svg', dpi=3000)

## Read the backlog
https://github.com/microsoft/azure-devops-python-api

In [ ]:
! pip install azure-devops

In [15]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint

# Fill in with your personal access token and org URL
personal_access_token = 'hzakqwyey6gejkpooxbhkcbwp2jequiobwzzfyrtpqrxssd2thra'
organization_url = 'https://dev.azure.com/lanteksms'

# Create a connection to the org
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get a client (the "core" client provides access to projects, teams, etc)
core_client = connection.clients.get_core_client()

# Get the first page of projects
get_projects_response = core_client.get_projects()
print(get_projects_response)
index = 0
while get_projects_response is not None:
    for project in get_projects_response:
        pprint.pprint("[" + str(index) + "] " + project.name)
        index += 1
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)
    else:
        # All projects have been retrieved
        get_projects_response = None

[<azure.devops.v7_0.core.models.TeamProjectReference object at 0x0000025533EF6160>, <azure.devops.v7_0.core.models.TeamProjectReference object at 0x0000025533EF65E0>, <azure.devops.v7_0.core.models.TeamProjectReference object at 0x0000025533EF6D30>, <azure.devops.v7_0.core.models.TeamProjectReference object at 0x0000025533EF6550>, <azure.devops.v7_0.core.models.TeamProjectReference object at 0x0000025533EF6370>]
'[0] DeustoCatedra'
'[1] Merlin'
'[2] Analytics'
'[3] Lantek 360'
'[4] CADCAM Cloud'


AttributeError: 'list' object has no attribute 'continuation_token'

In [18]:
# Define the connection parameters
personal_access_token = 'hzakqwyey6gejkpooxbhkcbwp2jequiobwzzfyrtpqrxssd2thra'
organization_name = 'lanteksms'
project_name = 'Merlin'
team_name = 'Merlin Team'

from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication

# Create a connection object
credentials = BasicAuthentication('', personal_access_token)
organization_url = f'https://dev.azure.com/{organization_name}'
connection = Connection(base_url=organization_url, creds=credentials)

# Get the work item tracking client
wit_client = connection.clients.get_work_item_tracking_client()

# Define the list of work item IDs
work_item_ids = [1862]

# Get the work items
work_items = wit_client.get_work_items(ids=work_item_ids)

# Print the ID and title of each work item
print("Work items:")
for work_item in work_items:
    print(f"- {work_item.id}: {work_item.fields['System.Title']}")

AzureDevOpsServiceError: TF401232: Work item 1862 does not exist, or you do not have permissions to read it.

In [ ]:
# Import the required modules
import requests
import json

# Define the personal access token and organization URL
personal_access_token = 'hzakqwyey6gejkpooxbhkcbwp2jequiobwzzfyrtpqrxssd2thra'
organization_url = 'https://dev.azure.com/lanteksms'

# Define the project name and the work item types to query
project = 'Merlin'
work_item_types = ['Epic', 'Feature', 'User Story']

# Define the query parameters
query_params = {
    'api-version': '7.1',
    'query': f"Select [System.Id], [System.Title], [System.WorkItemType], [System.State] From WorkItems Where [System.TeamProject] = '{project}' And [System.WorkItemType] In ({', '.join(work_item_types)}) And [System.State] <> 'Closed' order by [System.ChangedDate] desc"
}

# Create the request header with the authorization token
request_header = {
    'Authorization': f'Basic {personal_access_token}',
    'Content-Type': 'application/json'
}

# Send the request to the Azure DevOps REST API
response = requests.post(f'{organization_url}/_apis/wit/wiql?api-version=7.1', headers=request_header, data=json.dumps(query_params))

# Check if the response is successful
if response.status_code == 200:
    # Parse the response as JSON
    response_data = response.json()

    # Get the work item IDs from the response
    work_item_ids = [item['id'] for item in response_data['workItems']]

    # Print the number of work items found
    print(f'Found {len(work_item_ids)} work items in the backlog of {project} project.')

    # If there are any work item IDs, get their details
    if work_item_ids:
        # Create a comma-separated list of work item IDs
        work_item_ids_str = ','.join(map(str, work_item_ids))

        # Send another request to get the work item details
        response = requests.get(f'{organization_url}/_apis/wit/workitems?ids={work_item_ids_str}&$expand=relations&api-version=7.1', headers=request_header)

        # Check if the response is successful
        if response.status_code == 200:
            # Parse the response as JSON
            response_data = response.json()

            # Loop through the work items in the response
            for work_item in response_data['value']:
                # Get the work item ID, title, type, and state
                work_item_id = work_item['id']
                work_item_title = work_item['fields']['System.Title']
                work_item_type = work_item['fields']['System.WorkItemType']
                work_item_state = work_item['fields']['System.State']

                # Print the work item details
                print(f'Work item {work_item_id}: {work_item_title} ({work_item_type}, {work_item_state})')

                # If the work item has any relations, print them as well
                if 'relations' in work_item:
                    print('Related work items:')
                    for relation in work_item['relations']:
                        # Get the relation type and URL
                        relation_type = relation['rel']
                        relation_url = relation['url']

                        # Extract the related work item ID from the URL
                        related_work_item_id = relation_url.split('/')[-1]

                        # Print the relation details
                        print(f'- {relation_type}: {related_work_item_id}')
else:
    # Print the error message if the response is not successful
    print(f'Error: {response.reason}')
